# NCAA ML Model

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import csv
import sys
import os
from sklearn import svm

## Goals

Given the regular season game data predict the champion for the NCAA tournament this season

## Define the problem

Taking raw dataset from past games and predict a winner and take into account  that upsets do happen often in tournaments

https://www.breakthroughbasketball.com/stats/how-we-use-stats-Hagness.html

## Import Dataset

In [28]:
label_path = "./data/kaggle_2019_data/NCAATourneyDetailedResults.csv"
input_path = "./data/kaggle_2019_data/RegularSeasonDetailedResults.csv"
team_id_path = "./data/kaggle_2019_data/Teams.csv"
seed_path = "./data/kaggle_2019_data/NCAATourneySeeds.csv"
label_df = pd.read_csv(label_path)
input_df = pd.read_csv(input_path)
team_df = pd.read_csv(team_id_path)
seed_df = pd.read_csv(seed_path)
df = pd.concat([input_df],axis=1)
pd.set_option('display.max_rows', None)
df.head().transpose()

0     1     2     3     4
Season   2003  2003  2003  2003  2003
DayNum     10    10    11    11    11
WTeamID  1104  1272  1266  1296  1400
WScore     68    70    73    56    77
LTeamID  1328  1393  1437  1457  1208
LScore     62    63    61    50    71
WLoc        N     N     N     N     N
NumOT       0     0     0     0     0
WFGM       27    26    24    18    30
WFGA       58    62    58    38    61
WFGM3       3     8     8     3     6
WFGA3      14    20    18     9    14
WFTM       11    10    17    17    11
WFTA       18    19    29    31    13
WOR        14    15    17     6    17
WDR        24    28    26    19    22
WAst       13    16    15    11    12
WTO        23    13    10    12    14
WStl        7     4     5    14     4
WBlk        1     4     2     2     4
WPF        22    18    25    18    20
LFGM       22    24    22    18    24
LFGA       53    67    73    49    62
LFGM3       2     6     3     6     6
LFGA3      10    24    26    22    16
LFTM       16     9    14     8    17
LFTA       22    20    23    15    27
LOR        10    20    31    17    21
LDR        22    25    22    20    15
LAst        8     7     9     9    12
LTO        18    12    12    19    10
LStl        9     8     2     4     7
LBlk        2     6     5     3     1
LPF        20    16    23    23    14

## Feature Selection

In [30]:
features = ["Season","WTeamID","LTeamID","WScore","LScore","WLoc","WFGM","WFGA","LFGM","LFGA","WAst","LAst","WTO","LTO"]
new_df = df[features]
# The inputs
new_df.transpose()

0     1     2     3     4     5     6     7     8     9      ...  \
Season   2003  2003  2003  2003  2003  2003  2003  2003  2003  2003  ...   
WTeamID  1104  1272  1266  1296  1400  1458  1161  1186  1194  1458  ...   
LTeamID  1328  1393  1437  1457  1208  1186  1236  1457  1156  1296  ...   
WScore     68    70    73    56    77    81    80    75    71    84  ...   
LScore     62    63    61    50    71    55    62    61    66    56  ...   
WLoc        N     N     N     N     N     H     H     N     N     H  ...   
WFGM       27    26    24    18    30    26    23    28    28    32  ...   
WFGA       58    62    58    38    61    57    55    62    58    67  ...   
LFGM       22    24    22    18    24    20    19    20    24    23  ...   
LFGA       53    67    73    49    62    46    41    59    52    52  ...   
WAst       13    16    15    11    12    12    14    19     9    11  ...   
LAst        8     7     9     9    12     8    11    10    13    10  ...   
WTO        23    13    10    12    14     9    17    19    17     6  ...   
LTO        18    12    12    19    10    19    30    15    25    18  ...   

        82031 82032 82033 82034 82035 82036 82037 82038 82039 82040  
Season   2018  2018  2018  2018  2018  2018  2018  2018  2018  2018  
WTeamID  1411  1420  1426  1437  1438  1153  1172  1209  1246  1335  
LTeamID  1115  1436  1418  1344  1314  1222  1348  1426  1397  1217  
WScore     84    65    71    76    71    56    58    74    77    68  
LScore     69    62    68    66    63    55    57    61    72    65  
WLoc        N     A     N     N     N     N     N     N     N     N  
WFGM       26    25    25    28    21    20    19    25    25    24  
WFGA       44    56    55    64    50    46    50    56    50    52  
LFGM       25    18    25    23    20    19    20    17    23    23  
LFGA       63    38    61    61    49    55    52    57    62    55  
WAst       20    13    14    11    15     8     9     9     9    10  
LAst       11     8    15    16    15    12    12     9    14     6  
WTO        15     4    14    10     4    13    12     8    10     7  
LTO         8    13     7    11     9    12    14    13    10     8  

[14 rows x 82041 columns]

## Transform Data

In [53]:
# getting seed to match their WTeamID or LTeamID
seed_df.head().transpose()
WSeeds = []
LSeeds = []
for index, row in new_df.iterrows():
    Wquery = "Season == {} and TeamID == {}".format(str(row['Season']),str(row["WTeamID"]))
    Lquery = "Season == {} and TeamID == {}".format(str(row['Season']),str(row["LTeamID"]))
    Wval = seed_df.query(Wquery)["Seed"].values
    Lval = seed_df.query(Lquery)["Seed"].values
    Wval = Wval[0] if len(Wval) > 0 else 0
    Lval = Lval[0] if len(Lval) > 0 else 0
    print(Wval,Lval)
    WSeeds.append(Wval)
    LSeeds.append(Lval)
print(WSeeds)

Y10 W01
Z07 W03
Y03 0
0 0
X01 0
Y05 0
Z14 0
0 0
0 0
Y05 0
Z06 0
0 0
Y16 0
Z05 0
0 0
0 0
Z14 0
Z06 0
0 0
0 0
Z05 Y16
0 0
X04 0
0 0
X03 0
Z14 0
0 0
X02 0
Z02 Y14
0 Y15
0 0
X04 X03
X02 0
0 0
Y10 0
Z10 0
0 0
0 0
W10 0
W13 Z07
0 0
0 0
W12 0
X12 0
Z11 0
0 0
0 0
0 0
0 0
0 0
X10 0
0 0
0 0
0 0
0 0
0 0
0 0
Z09 0
0 0
0 0
0 0
0 0
0 0
Z02 0
0 0
0 0
0 0
X08 0
0 0
Y03 0
0 0
X07 X16b
Y06 0
0 0
W09 0
0 0
0 0
0 0
0 0
Z05 0
W01 0
W06 0
0 0
0 0
X09 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 X15
0 0
0 X11
0 0
0 0
0 0
0 0
Y09 0
0 0
Z16 0
0 0
0 0
Y12 0
0 0
Z01 Z13
0 0
0 0
0 0
X12 0
W08 0
0 0
Z08 0
X05 0
0 0
0 0
Z03 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
W04 0
0 0
Y03 0
0 0
0 0
0 0
W01 0
W11 0
Y02 0
0 0
0 0
0 0
X13 0
0 0
0 0
W07 0
0 0
X01 0
Y13 0
0 W05
0 0
Z15 0
Z16 0
0 0
0 0
0 0
0 0
Y12 0
0 0
0 0
Y05 Z12
0 0
0 0
0 0
0 0
0 0
0 0
X10 0
0 0
Y04 0
0 0
X02 0
0 0
0 0
0 0
0 0
0 0
Z04 0
0 0
0 0
X08 0
X06 0
0 X16b
0 0
0 0
Y08 0
0 0
0 0
0 0
Y11 0
0 0
0 0
0 W15
0 0
X04 0
W03 0
0 X16a
0 0
Y13 0
0 0
0 0
0 0
0

X16b 0
Z16 0
0 0
0 0
0 0
0 W15
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Z02 0
0 0
0 0
0 0
0 0
0 0
0 0
Y02 Z05
0 0
0 0
W16 0
0 0
0 0
X14 0
X16a 0
Y09 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
X10 Z14
0 0
X02 W05
0 0
0 0
Y14 0
Z04 0
0 Y16
W14 0
Y03 0
Z07 0
Y06 0
0 0
0 0
0 0
W01 X05
0 0
0 0
0 X03
W02 0
0 0
0 0
Y10 0
W10 0
0 0
0 0
0 0
Z11 0
Z08 0
0 0
Z06 0
0 0
Y04 0
0 0
0 0
Z03 0
0 0
0 0
0 0
0 0
0 0
Y07 0
0 0
0 0
0 0
0 0
Y01 0
W04 0
X08 0
0 0
X06 0
0 Y05
0 0
0 0
0 0
0 0
0 0
0 0
0 0
W06 0
0 X11
W11 0
0 0
0 0
0 0
Y11 0
0 X13
0 0
0 0
W07 0
W03 0
0 0
X16b 0
0 0
Z16 0
0 0
Y15 0
Y12 X12
0 0
Z01 0
Z10 0
0 0
W12 0
W08 Y08
0 0
0 0
0 0
Z09 0
0 0
0 Z12
0 0
0 0
0 0
0 0
X07 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
X04 0
0 X15
0 W13
X14 0
0 0
0 0
0 0
Z15 0
Z13 0
0 0
0 0
0 0
W14 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Z01 0
Z10 0
0 0
W10 0
W13 0
0 0
0 0
0 Z16
0 0
0 0
W12 0
X12 0
Z11 0
W08 0
0 0
0 0
0 0
0 0
Z08 0
0 0
0 0
0 0
0 0
X05 0
Z06 0
0 0
0 0
0 0
Y04 0
0 0
0 0
0 0
0 0
0 0
W15 0
0 0
0 0
0 0
X02 0
0 0
0 0
0 0
0 0
0

0 0
W13 0
0 0
0 W06
0 0
0 0
0 0
0 0
0 Y11
W12 0
Z11 0
0 0
W08 0
0 Z08
0 0
0 0
0 0
X10 0
0 0
0 0
0 0
0 0
Y04 W07
0 0
0 0
0 0
0 0
Z03 0
0 0
0 0
0 0
W15 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Z09 0
0 0
Y14 0
0 0
0 0
0 0
Y16 0
0 X16a
0 0
0 0
Y01 X08
0 0
0 0
0 0
W04 Y03
0 0
0 0
Z07 0
0 X14
0 0
0 0
X07 0
W05 0
Y06 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 Z16
0 0
0 0
W01 0
0 0
Y08 0
W11 0
0 0
0 0
0 0
X09 Z04
0 0
0 0
0 0
0 0
X15 0
0 0
X13 0
0 0
0 0
0 Y02
0 0
0 0
0 0
0 0
0 0
0 0
0 0
X04 0
W03 Z05
0 0
0 W09
0 X02
X01 0
0 0
Y13 0
0 0
0 0
0 0
0 0
0 0
X11 0
0 0
Y09 0
Z15 0
0 0
0 0
0 X05
0 0
0 0
Z13 0
0 0
Y12 0
0 0
Z12 0
0 Z06
0 0
Y05 Y07
0 0
0 0
X03 0
0 0
0 0
0 0
0 W14
Z02 0
0 0
0 0
0 0
0 0
X12 0
0 0
X02 0
0 0
0 X16a
0 0
0 0
0 0
0 0
X06 W02
0 0
0 0
0 Z14
0 0
0 0
X15 0
0 0
X01 0
Y09 0
0 0
0 0
0 0
0 0
0 0
0 0
W15 0
0 Y10
0 0
0 0
0 0
0 0
Z04 X07
0 0
0 0
Y06 0
0 0
W09 0
0 0
Z05 0
Y02 0
0 0
0 0
0 0
0 0
0 Z08
W03 0
0 0
0 0
0 0
0 0
Y15 0
0 0
0 0
0 0
0 0
0 0
Z11 0
X05 0
Z06 0
Y04 0
0 W16
0 0
Z03 X06
0 0
0 0
0 X16b
0

X12 0
0 0
W04 Z15
0 0
0 0
0 0
0 0
Y05 Z16a
0 0
0 0
0 0
0 0
0 0
Z13 0
0 0
0 0
0 0
0 0
0 0
0 0
X04 Y06
0 X15
0 0
0 X03
0 0
W15 0
W06 W05
Z14 0
0 0
0 0
W14 0
Z05 0
Y01 0
0 0
0 0
Y04 0
0 0
0 0
0 Y12
0 0
X08 Y09
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
X10 0
0 0
0 0
0 0
W01 Z07
W13 0
0 0
0 0
Z03 0
Z02 Z08
0 0
0 0
0 0
0 0
Y07 0
W02 0
0 0
W12 0
0 0
0 0
0 0
0 0
0 0
Y02 X16
0 0
Y03 0
0 0
0 0
0 0
Y11 Y13
0 0
X09 0
0 0
0 0
0 0
0 0
Y10 0
0 0
0 0
X05 0
0 0
Y08 X13
0 0
Z09 0
0 0
0 0
0 0
Z11 0
X06 0
0 0
0 0
0 0
0 0
0 0
Y05 Y14
0 0
W10 0
0 0
0 0
0 0
0 0
0 W16
0 0
0 Z10
W03 0
0 0
0 0
0 0
0 0
0 0
0 0
0 Z16a
0 0
0 Y15
0 0
0 0
0 0
0 Z13
0 0
0 0
0 0
0 W14
X15 0
0 0
0 0
0 0
X08 0
0 0
W09 0
0 0
0 0
0 0
0 0
Z06 0
0 0
0 0
X12 Y02
0 0
0 0
0 0
0 0
Y09 0
0 0
0 0
0 0
X02 0
0 0
0 0
X10 0
0 0
0 0
W01 0
0 0
W13 0
Y05 0
0 0
0 Z14
0 0
0 0
0 0
Z03 0
Z02 X04
0 0
0 0
0 0
Z13 Y15
W05 0
0 0
0 0
0 0
0 0
0 0
Z01 0
Z16b 0
0 0
Y07 0
0 0
0 0
0 0
W02 Z09
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 Z07
0 0
0 0
0 0
Z12 Z10
0 0
Y03

0 0
X09 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Y10 0
0 0
0 0
0 0
0 0
Y01 W07
0 0
X01 0
0 0
X05 0
0 0
0 0
W03 0
Y08 0
0 0
0 0
0 0
0 0
0 0
Y14 0
X14 0
0 0
0 0
0 0
0 0
X16 0
Z11 0
0 0
X13 0
Y13 0
Z15 0
X06 Y05
X15 0
0 0
X11 0
Z08 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Y12 0
0 0
0 0
0 0
0 0
X12 0
0 0
0 0
0 0
0 0
0 0
0 0
X02 Y03
0 0
0 0
W13 0
0 0
0 0
Z16a 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
W15 0
0 0
0 0
0 0
0 0
Z05 0
0 0
0 0
0 0
0 0
0 0
Y08 0
0 0
0 0
0 0
Y14 0
Z11 0
0 0
0 0
0 0
0 0
Z06 X03
0 0
Y09 0
X07 0
0 0
Z03 Y04
0 0
Z01 0
Y16 0
Y07 0
W12 0
0 0
0 0
W08 X05
W03 0
Z09 0
0 0
W11 0
X08 0
0 0
0 0
0 0
0 0
X10 0
0 0
0 0
W01 X04
0 0
W13 0
0 0
0 0
W05 0
0 0
0 0
0 X11
W10 W04
Y12 0
0 0
0 0
Z07 0
0 0
W02 Y05
0 0
0 0
Z14 0
Y02 0
Z05 0
0 0
0 0
X09 0
0 0
Y01 0
0 0
0 0
0 X16
0 0
0 0
Y13 0
X15 0
0 0
0 0
0 0
Y06 0
0 Z13
W07 0
W09 0
0 0
0 0
0 Y16
0 0
0 0
0 0
0 W06
0 0
0 X13
0 0
0 0
0 0
0 0
0 0
0 0
0 W15
0 0
0 0
0 0
Z10 0
0 0
0 0
0 0
0 0
0 0
0 0
Z12 0
0 0
Y11 Z04
0 0
0 0
0 0
0 0
0 0
0 0
X01 0
0 0
0 0
0 0
0 0

...

0 0
0 0
W02 0
0 0
0 0
0 0
0 0
X14 0
W04 Z02
0 0
0 0
0 0
0 0
0 0
0 0
0 0
X01 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Y05 Y01
Y15 0
0 0
Z11a W03
0 0
Z14 0
0 0
0 0
0 0
0 0
0 0
0 0
Y02 0
0 0
0 0
0 0
W10 0
0 0
Y08 0
Z05 0
0 X09
0 0
0 Z08
0 0
0 0
0 0
0 0
0 0
Z15 0
Y16a 0
0 0
W14 0
W16b 0
0 0
Z01 X05
X15 0
0 0
0 0
0 0
0 0
0 Y07
Y10 X04
Y03 X02
0 0
X12 0
Y04 X06
0 0
0 0
0 0
0 0
0 0
X16 0
0 0
0 0
0 0
0 0
0 0
0 0
Z09 0
0 0
W06 0
0 W15
W07 0
0 0
0 0
0 0
0 0
X07 0
0 0
0 0
0 W05
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Z16 0
Y16b 0
0 0
0 0
Z03 0
0 0
0 0
0 0
W12 0
W11b 0
0 0
0 0
0 Z12
X10 0
Y13 0
W01 0
0 0
0 0
Z11b 0
0 0
0 0
Z10 0
Z13 0
0 0
0 0
X11 Y06
0 0
0 0
0 0
X03 0
0 0
0 0
W08 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
X13 0
0 0
0 0
0 0
0 0
0 0
Y01 Z11a
0 0
0 0
Y16a 0
0 0
0 0
0 0
0 0
Y13 0
W05 Y02
0 W13
0 0
0 W07
0 0
0 Z14
0 0
Z04 W10
0 0
0 0
Z07 0
W04 0
0 X08
0 0
Z02 0
0 0
Y07 Y09
X05 Z11b
0 X06
X12 0
Y11 0
0 0
0 Y14
0 0
0 0
0 Y05
0 0
X09 Z08
W01 0
0 0
X02 0
W03 Y10
0 0
0 0
Z06 0
0 0
0 0
0 0
X03 0
0 0
0 0
0 0
0 0
0 0
0 0
0 X13
Y

0 0
Z06 0
0 Z16a
0 0
0 0
0 0
X01 0
Y09 0
Y05 0
0 0
0 0
Y11 Y14
0 0
0 0
W13 0
0 0
0 0
Y06 0
0 0
X03 0
0 0
0 0
0 0
Z08 0
0 0
0 X14
0 0
0 0
0 0
X09 0
Y07 0
0 W11a
0 0
Z02 0
0 0
0 0
0 0
0 0
Z05 0
0 0
0 0
0 0
0 0
0 0
Z10 0
W02 0
X07 0
0 0
Z12 0
0 0
X08 0
0 Y16
0 0
0 0
0 0
0 0
0 0
X11b 0
X06 0
Y03 0
Y10 0
W03 0
0 W16a
0 0
0 0
0 0
0 W15
0 0
0 0
0 0
W01 0
Y01 Z13
W08 0
0 0
0 0
W04 0
0 0
0 0
Z01 0
0 0
0 0
Y13 0
0 0
X02 0
0 0
0 X16
0 0
0 0
0 0
0 0
Z03 0
0 0
0 0
0 0
0 0
0 0
0 0
W14 0
0 0
Z07 W05
0 0
W11b 0
0 0
0 0
0 0
0 0
Y04 Y16
W07 X14
0 0
0 0
W10 0
0 0
X05 0
Y08 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Y05 0
Z15 0
0 0
0 0
Y06 0
X09 0
Z05 W16b
X10 0
0 0
W02 0
0 0
0 0
X08 0
0 0
0 0
0 0
0 0
0 0
W08 0
0 0
0 Z16b
0 0
0 0
0 0
0 0
0 0
0 0
Y02 0
0 0
0 0
0 0
0 Z16a
0 0
W06 0
0 W16a
0 0
0 0
0 0
0 0
0 0
0 0
0 X16
0 0
W13 0
Z03 0
0 Z10
0 0
Z08 0
Z14 0
0 0
0 W12
0 0
Y07 X07
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 X12
X06 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Y01 0
0 0
W04 X13
0 0
Z01 0
0 0
W09 Z15
X11a 

0 0
X14 0
W10 0
0 0
0 0
0 0
Y02 0
X05 0
X13 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
X15 0
0 0
0 0
0 0
Z08 W14
0 Y14
Z14 0
0 0
X09 0
Y07 0
0 0
0 0
0 0
Z05 0
X10 0
0 0
0 0
0 0
0 0
0 0
Z12 0
0 0
0 W16b
0 0
0 0
0 0
X11b Y13
Z07 0
W03 0
0 0
W11b 0
0 0
0 Z13
0 0
0 0
Y01 0
W08 0
0 0
0 0
W04 0
Z01 W13
0 0
0 0
0 0
0 0
Y08 0
0 0
X02 0
W06 0
0 0
0 0
Y15 0
0 0
Z06 Z10
0 0
Y09 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
X08 0
W11a 0
0 0
Y03 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
W05 0
0 Z02
Y04 0
0 0
W10 0
0 0
0 0
Y02 0
0 0
0 0
Z09 0
0 0
0 0
0 0
0 0
0 0
X01 0
0 0
0 0
Z15 0
0 0
0 0
0 0
0 0
Z03 0
X03 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
W02 0
X07 X15
Z11 Z04
0 0
0 0
0 0
Z07 Y13
0 0
0 0
0 0
0 0
0 0
0 0
X11a 0
0 0
0 0
X14 0
0 0
0 0
X05 0
X13 0
W15 0
0 0
0 0
0 0
0 0
0 0
0 0
W06 0
0 0
0 0
0 0
0 0
0 0
0 0
W16a 0
0 0
0 0
W13 0
0 0
Y06 0
0 0
0 Y11
0 0
0 0
0 0
W12 0
0 0
X09 0
0 0
Y07 0
0 0
X12 Y12
0 0
X10 0
0 0
0 0
0 0
Z10 0
W16b 0
0 0
Z12 0
0 0
W11a X11b
0 0
0 0
0 0
X06 0
Y10 W09
0 0
W03 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
W01 0
Y01

Z04 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
X15 0
0 0
X01 Z07
Y09 0
0 0
0 0
Y05 W05
0 0
0 0
Z15 0
0 0
W16a 0
0 0
0 0
0 0
0 Z08
0 0
Z14 0
0 0
0 0
0 0
0 0
W12 0
0 0
0 0
0 Z16a
X09 Z02
0 0
0 0
0 0
X12 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
X16 0
0 0
0 0
0 0
X07 0
Z12 0
0 0
0 0
0 0
0 0
0 0
0 0
W14 0
0 0
0 0
0 0
X11b 0
0 0
Y03 0
0 0
0 0
Y10 0
W03 0
0 0
0 0
Z16b 0
0 0
W11b 0
0 0
0 0
Y16 0
0 0
Z13 0
0 Z11
0 0
0 0
0 0
0 0
0 X06
Y01 X02
W08 0
0 0
0 W13
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
X14 0
X05 0
Z06 0
Y11 0
X03 0
0 0
W02 0
0 Y12
X08 0
0 0
W11a 0
0 0
0 0
0 0
W01 0
0 0
0 0
W04 0
Y14 0
0 0
0 Z13
0 0
X02 0
0 0
0 X15
0 0
0 0
0 0
0 0
0 0
X01 Y09
0 0
Z15 0
Z03 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
Z16b 0
0 0
X04 0
0 0
0 0
0 0
X05 Z02
0 W06
0 0
0 0
0 0
0 Y13
Y05 0
0 0
0 0
Z05 0
X10 0
X07 0
Z12 0
0 0
0 0
X06 0
Z07 W07
0 0
Z01 0
0 0
Y04 0
0 0
0 Y11
X14 0
W10 0
0 0
Y02 Z06
Y12 0
0 0
0 0
0 X11b
0 0
0 0
0 W05
0 0
0 0
0 0
Y06 0
X03 0
0 0
Z08 W09
0 0
Y07 0
0 0
W02 0
0 0
X08 Z10
W14 0
W11a 0
0 0
0 0
Y03 0
W03 Y10
0 0
0 0

0 0
X08 W10
W14 0
0 0
0 0
W11a 0
0 0
0 0
0 X11a
0 0
X11b X05
0 0
Y03 0
Y10 W05
Z07 W09
W03 X06
Z16b 0
0 0
0 0
W11b 0
Y16 0
Z13 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
W01 0
Y01 0
0 0
0 0
0 0
Y14 0
Z01 0
0 0
X14 0
Y02 W04
X13 0
0 0
0 0
0 0
0 0
Z06 0
X15 0
Z15 0
Y11 0
0 0
Z03 W02
0 0
0 0
0 0
W16b 0
0 0
0 0
0 0
Z13 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
X13 0
Z04 0
X15 0
0 0
0 0
0 0
0 0
0 0
Z12 0
0 0
Z13 0
Y14 0
0 0
0 0
X13 0
0 0
Z04 0
W16a 0
0 0
0 0
Z16a 0
0 0
0 0
0 0
0 0
Z12 0
0 0
0 0
X11b 0
Z16b 0
Y16 0
0 0
Y14 0
0 0
0 X09
X14 0
0 0
0 X11a
0 0
0 0
0 0
0 0
0 Z09
0 0
0 0
0 0
0 0
Z02 X11b
0 W08
0 X10
0 0
0 0
0 0
0 0
0 0
0 0
Y10 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
W09 Z07
Y04 0
W07 0
Y13 0
W10 X08
X05 0
W15 0
X02 0
0 0
0 0
0 0
0 Z08
0 0
X01 0
Y09 X06
0 0
W13 0
0 0
0 0
Z14 0
0 0
0 0
Z16a 0
Y07 0
0 0
X12 0
Z02 Y06
0 0
0 0
Z10 Y08
0 0
0 0
Z11 0
0 0
0 0
W14 0
0 0
0 0
0 0
0 0
0 0
W03 Y10
0 0
0 0
0 0
0 0
W11b 0
0 0
0 0
0 0
0 0
W01 0
Y01 0
0 0
W05 0
Z01 0
W09 X04
Y04 W11b
0 0
W07 W06
Y13 0
Y02 0
W15 0
Y12 0
0 0
0

In [56]:
new_df = new_df.assign(WSeed = WSeeds)
new_df = new_df.assign(LSeed = LSeeds)

new_df.head().transpose()

0     1     2     3     4
Season   2003  2003  2003  2003  2003
WTeamID  1104  1272  1266  1296  1400
LTeamID  1328  1393  1437  1457  1208
WScore     68    70    73    56    77
LScore     62    63    61    50    71
WLoc        N     N     N     N     N
WFGM       27    26    24    18    30
WFGA       58    62    58    38    61
LFGM       22    24    22    18    24
LFGA       53    67    73    49    62
WAst       13    16    15    11    12
LAst        8     7     9     9    12
WTO        23    13    10    12    14
LTO        18    12    12    19    10
WSeed     Y10   Z07   Y03     0   X01
LSeed     W01   W03     0     0     0

## Model definition

## Trainning

## Result